## Linear Regression on Boston Housing Dataset

We will take the Housing dataset which contains information about different houses in Boston. There are 506 samples and 13 feature variables in this dataset. 

The objective is to predict the value of prices of the house using the given features.

In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# define column names
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

In [3]:
# load data, check pandas.read_csv usage: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
boston = pd.read_csv('E:\Sebnewrepo\Leo Study\week5\housing.csv', delim_whitespace=True, names=names)

FileNotFoundError: [Errno 2] File housing.csv does not exist: 'housing.csv'

In [ ]:
print("Boston housing dataset has {} data points with {} variables each.".format(*boston.shape))

In [ ]:
boston.head(10)

### The features can be summarized as follows:

CRIM: Per capita crime rate by town<br />
ZN: Proportion of residential land zoned for lots over 25,000 sq. ft<br />
INDUS: Proportion of non-retail business acres per town<br />
CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)<br />
NOX: Nitric oxide concentration (parts per 10 million)<br />
RM: Average number of rooms per dwelling<br />
AGE: Proportion of owner-occupied units built prior to 1940<br />
DIS: Weighted distances to five Boston employment centers<br />
RAD: Index of accessibility to radial highways<br />
TAX: Full-value property tax rate per 10,000 dollars<br />
PTRATIO: Pupil-teacher ratio by town<br />
B: the proportion of people of African American descent by town<br />
LSTAT: Percentage of lower status of the population<br />
MEDV: Median value of owner-occupied homes in $1000s<br />

The prices of the house indicated by the variable MEDV is our target variable and the remaining are the feature variables based on which we will predict the value of a house.

In [ ]:
print(boston.describe())

### Data preprocessing

After loading the data, it’s a good practice to see if there are any missing values in the data. We count the number of missing values for each feature using isnull()

In [ ]:
boston.isnull().sum()

### Exploratory Data Analysis

Exploratory Data Analysis is a very important step before training the model. In this section, we will use some visualizations to understand the relationship of the target variable with other features.

Let’s first plot the distribution of the target variable MEDV. We will use the distplot function from the seaborn library.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.distplot(boston['MEDV'], bins=30)
plt.show()

We see that the values of MEDV are distributed normally with few outliers. You can also use the histogram plot function from the matplotlib library.

In [ ]:
plt.hist(boston['MEDV'], bins=30)
plt.xlabel("House prices in $1000")
plt.show()

Let's see how these features plus MEDV distributions looks like

In [ ]:
boston.describe()

In [ ]:
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in boston.items():
    sns.distplot(v, ax=axs[index], kde_kws={'bw':0.1})
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

Next, we create a correlation matrix that measures the linear relationships between the variables. The correlation matrix can be formed by using the corr function from the pandas dataframe library. We will use the heatmap function from the seaborn library to plot the correlation matrix.

In [ ]:
correlation_matrix = boston.corr().round(2)
correlation_matrix

In [ ]:
# annot = True to print the values inside the square
sns.heatmap(data=correlation_matrix, annot=True)

The correlation coefficient ranges from -1 to 1. If the value is close to 1, it means that there is a strong positive correlation between the two variables. When it is close to -1, the variables have a strong negative correlation.

### Observations:

To fit a linear regression model, we select those features which have a high correlation with our target variable MEDV. By looking at the correlation matrix we can see that RM has a strong positive correlation with MEDV (0.7) where as LSTAT has a high negative correlation with MEDV(-0.74).

An important point in selecting features for a linear regression model is to check for multi-co-linearity. The features RAD, TAX have a correlation of 0.91. These feature pairs are strongly correlated to each other. We should not select both these features together for training the model. Same goes for the features DIS and AGE which have a correlation of -0.75.

In [ ]:
# statsmodels lib can generate a summary of linear regression

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Filter the result by MEDV < 50
boston = boston[boston['MEDV'] < 50]

# split the raw data into train and test
x = boston.iloc[:,0:13]
y = boston.iloc[:,13]
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('train set shape')
print(x_train.shape, y_train.shape)
print('test set shape')
print(x_test.shape, y_test.shape)

def ols_lr(x,y):
    x = sm.add_constant(x) # add intercept
    model = sm.OLS(y,x).fit()
    print(model.summary())
    return model

def eval(y_test, y_pred):
    print('MAE')
    print(metrics.mean_absolute_error(y_test, y_pred))
    print('MSE')
    print(metrics.mean_squared_error(y_test, y_pred))
    plt.plot(y_test, y_pred,'o')
    plt.xlabel('true')
    plt.ylabel('predicted')
    return

#### Prediction without Log Transformation:
For feature selection, backward elimination is used here to reduce the predictors. So, fristly try not to eliminate the highly related pairs of variables.
##### 01. Use all predictors

In [ ]:
model1 = ols_lr(x_train,y_train)
x_test1 = sm.add_constant(x_test)
y_pred = model1.predict(x_test1)

eval(y_test,y_pred)

##### 02. Get rid of INDUS and AGE
The P-Value for INDUS and AGE is too high, get rid of them.

In [ ]:
x_train1 = x_train.drop('INDUS',axis = 1)
x_train1 = x_train1.drop('AGE',axis = 1)
x_test1 = x_test.drop('INDUS',axis = 1)
x_test1 = x_test1.drop('AGE',axis = 1)


In [ ]:
# do linear regression again:
model1 = ols_lr(x_train1,y_train)

In [ ]:
x_test11 = sm.add_constant(x_test1)
y_pred1 = model1.predict(x_test11)

eval(y_test,y_pred1)

##### 02. Get rid of other predictors
Now, get rid of one of the predictors in the pairs of RAD - TAX, DIS - NOX which have high correlation value

Try to get rid of RAD first:

In [ ]:
x_train2 = x_train1.drop('RAD',axis = 1)

x_test2 = x_test1.drop('RAD',axis = 1)

model1 = ols_lr(x_train2,y_train)

In [ ]:
x_test21 = sm.add_constant(x_test2)
y_pred2 = model1.predict(x_test21)

eval(y_test,y_pred2)

Get rid of TAX

In [ ]:
x_train3 = x_train1.drop('TAX',axis = 1)

x_test3 = x_test1.drop('TAX',axis = 1)
model1 = ols_lr(x_train3,y_train)

x_test31 = sm.add_constant(x_test3)
y_pred3 = model1.predict(x_test31)

eval(y_test,y_pred3)

In [ ]:
## get rid of DIS, the columns that are dropped are: DIS, TAX, INDUS, AGE
x_train4 = x_train3.drop('DIS',axis = 1)

x_test4 = x_test3.drop('DIS',axis = 1)

model1 = ols_lr(x_train4,y_train)
x_test41 = sm.add_constant(x_test4)
y_pred4 = model1.predict(x_test41)

eval(y_test,y_pred4)

In [ ]:
x_train5 = x_train4.drop('ZN',axis = 1)

x_test5 = x_test4.drop('ZN',axis = 1)

model1 = ols_lr(x_train5,y_train)
x_test51 = sm.add_constant(x_test5)
y_pred5 = model1.predict(x_test51)

eval(y_test,y_pred5)

#### Prediction with Log Transformation:
Some of the predictors have right skewed distribution, log transformation can be applied here to adjust the predictors' distribution.

In [ ]:
boston1 = pd.read_csv('housing.csv', delim_whitespace=True, names=names)

# Filter the result by MEDV < 50
boston1 = boston1[boston1['MEDV'] < 50]

boston1['CRIM'] = np.log(boston1['CRIM'])
boston1['DIS'] = np.log(boston1['DIS'])
boston1['LSTAT'] = np.log(boston1['LSTAT'])
boston1['RAD'] = np.log(boston1['RAD'])

fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in boston1.items():
    sns.distplot(v, ax=axs[index], kde_kws={'bw':0.1})
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
correlation_matrix1 = boston1.corr().round(2)
plt.figure(figsize=(13,11))
sns.heatmap(data = correlation_matrix1, annot=True)

In [ ]:
x = boston1.iloc[:,0:13]
y = boston1.iloc[:,13]
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('train set shape')
print(x_train.shape, y_train.shape)
print('test set shape')
print(x_test.shape, y_test.shape)

##### 01. Use all predictors

In [ ]:
model1 = ols_lr(x_train,y_train)
x_test1 = sm.add_constant(x_test)
y_pred = model1.predict(x_test1)

eval(y_test,y_pred)

##### 02. Get rid of CRIM, ZN, INDUS

In [ ]:
x_train1 = x_train.drop('INDUS',axis = 1)
x_train1 = x_train1.drop('ZN',axis = 1)
x_train1 = x_train1.drop('CRIM',axis = 1)
x_test1 = x_test.drop('INDUS',axis = 1)
x_test1 = x_test1.drop('ZN',axis = 1)
x_test1 = x_test1.drop('CRIM',axis = 1)

In [ ]:
model1 = ols_lr(x_train1,y_train)
x_test11 = sm.add_constant(x_test1)
y_pred1 = model1.predict(x_test11)

eval(y_test,y_pred1)

##### 02. Get rid of AGE, RM, TAX, DIS
Age has a big P-value

In [ ]:

x_train2 = x_train1.drop('AGE',axis = 1)

x_test2 = x_test1.drop('AGE',axis = 1)
model1 = ols_lr(x_train2,y_train)
x_test21 = sm.add_constant(x_test2)
y_pred2 = model1.predict(x_test21)

eval(y_test,y_pred2)

get rid of AGE, TAX

In [ ]:

x_train3 = x_train2.drop('TAX',axis = 1)

x_test3 = x_test2.drop('TAX',axis = 1)
model1 = ols_lr(x_train3,y_train)
x_test31 = sm.add_constant(x_test3)
y_pred3 = model1.predict(x_test31)

eval(y_test,y_pred3)

Get Rid of AGE,DIS

In [ ]:

x_train4 = x_train2.drop('DIS',axis = 1)

x_test4 = x_test2.drop('DIS',axis = 1)
model1 = ols_lr(x_train4,y_train)
x_test41 = sm.add_constant(x_test4)
y_pred4 = model1.predict(x_test41)

eval(y_test,y_pred4)

Get Rid of AGE,DIS, RM

In [ ]:
x_train5 = x_train4.drop('RM',axis = 1)

x_test5 = x_test4.drop('RM',axis = 1)
model1 = ols_lr(x_train5,y_train)
x_test51 = sm.add_constant(x_test5)
y_pred5 = model1.predict(x_test51)

eval(y_test,y_pred5)

get rid of AGE, RM, TAX, DIS

In [ ]:
x_train6 = x_train5.drop('TAX',axis = 1)

x_test6 = x_test5.drop('TAX',axis = 1)
model1 = ols_lr(x_train6,y_train)
x_test61 = sm.add_constant(x_test6)
y_pred6 = model1.predict(x_test61)

eval(y_test,y_pred6)

#### Conclusion



1. Do log transformation to some of the predictors with right-skewed distribution can improve the performance of the linear regression model.

2. Get rid of some of the predictors with high p-value can improve the performance.

3. Getting rid of some preditors to avoid multi co-linearity maybe will increase the value of MAE, MSE, which means the performance of the model gets worse.